In [26]:
import re
import time
import traceback
import threading
from IPython.display import clear_output

from chat_tools import *
from assistant import OpenAIAssistant

# Setup

In [27]:
# OpenAIAssistant
# With long term memory
# You must have Qdrant running on localhost:6333
# docker run -p 6333:6333 -v <your_save_dir_here>:/qdrant/storage qdrant/qdrant
api_key = ''
system_prompt = None
debug = False
use_long_term_memory = False
use_short_term_memory = False
use_knowledge_retrieval = False
summarize_short_term_memory = False
summarize_long_term_memory = False
summarize_knowledge_retrieval = False
short_term_memory_max_tokens = 750
long_term_memory_max_tokens = 500
knowledge_retrieval_max_tokens = 1000
short_term_memory_summary_max_tokens = 300
long_term_memory_summary_max_tokens = 300
knowledge_retrieval_summary_max_tokens = 600
long_term_memory_collection_name = 'long_term_memory'

assistant = OpenAIAssistant(api_key, system_prompt=system_prompt, long_term_memory_collection_name=long_term_memory_collection_name, use_long_term_memory=use_long_term_memory, use_short_term_memory=use_short_term_memory, memory_manager=None, debug=debug, summarize_short_term_memory=summarize_short_term_memory, summarize_long_term_memory=summarize_long_term_memory, short_term_memory_max_tokens=short_term_memory_max_tokens, long_term_memory_max_tokens=long_term_memory_max_tokens, short_term_memory_summary_max_tokens=short_term_memory_summary_max_tokens, long_term_memory_summary_max_tokens=long_term_memory_summary_max_tokens, use_knowledge_retrieval=use_knowledge_retrieval, summarize_knowledge_retrieval=summarize_knowledge_retrieval, knowledge_retrieval_max_tokens=knowledge_retrieval_max_tokens, knowledge_retrieval_summary_max_tokens=knowledge_retrieval_summary_max_tokens)
cache = {}

In [28]:
def parse_action(text, api_key='', action_dict={}, cache={}, max_tokens=500):
    try:
        # Regular expression pattern to match the last "Action" and "Action Input" in the text
        pattern = r'Action: ([^\n]*\n)+Action Input: ([^\n]*)$'

        match = re.search(pattern, text, re.MULTILINE)

        chatgpt = OpenAIAssistant(api_key, system_prompt=None, use_long_term_memory=False, use_short_term_memory=False, memory_manager=None, use_knowledge_retrieval=False)

        if match:
            last_action = match.group(1).strip().lower()
            last_action_input = match.group(2).strip()
            if cache.get(last_action+last_action_input):
                return cache[last_action+last_action_input]
            
            # Call the corresponding function based on the action name
            if last_action in action_dict:
                content = action_dict[last_action](last_action_input, chatgpt, max_tokens)
                cache[last_action+last_action_input] = content
                return content
            else:
                return 'Action does not exist!'
        else:
            return 'Invalid action format. Remember to include "Action:" and "Action Input:"'
    except Exception as e:
        # return the error message
        return '{error: ' + str(e) + '}'
    

def parse_action_with_timer(text, api_key, action_dict, cache={}, max_tokens=500, timeout=120.0):
    result = {'value': None}
    finished = threading.Event()

    def target():
        nonlocal result
        result['value'] = parse_action(text, api_key, action_dict, cache, max_tokens)
        finished.set()

    thread = threading.Thread(target=target)
    thread.start()
    finish = finished.wait(timeout)

    if not finish:
        return 'Error: Function execution timed out.'
    return result['value']

def web_search_with_timer(text, timeout=120.0):
    search = WebSearch()
    result = {'value': None}
    finished = threading.Event()

    def target():
        nonlocal result
        results = search.search(keywords=text, safesearch='Off', time=None, max_results=10, cache=True)
        out = '{'
        for result_ in results:
            out += 'title: ' + result_['title'] + ',\n\tbody: ' + result_['body'] + ',\n\t' + 'url: ' + result_['href'] + ',\n\t'
        result['value'] = out.strip() + '}'
        finished.set()

    thread = threading.Thread(target=target)
    thread.start()
    finish = finished.wait(timeout)

    if not finish:
        return 'Error: Function execution timed out.'
    return result['value']

In [62]:
def action_loop(assistant, user_prompt, cache={}, function_definitions=[], max_observation_tokens=500, memory_tokens=3000, max_loops=6, sleep=5, **kwargs):
    prompt = "Respond to the following prompt as best as you can. You have access to the following tools:\n"  + '\n'.join([f'{function_definition["name"]}: {function_definition["definition"]}' for function_definition in function_definitions])
    prompt = prompt + '\n' + """You must use the following format:
Prompt: The prompt you are responding to
Thought: You should always think about what to do
Action: Must be one of """
    prompt = prompt + '[' + ', '.join([f'{function_definition["name"]}' for function_definition in function_definitions]) + ']\n'
    prompt = prompt + """Action Input: The input to the action
Observation: the result of the action
... (this Thought/Action/Observation loop can repeat N times)
Thought: I now know the final answer
Final Answer: The final answer to the original input question
Begin!
Prompt: {prompt}"""

    action_dict = {function_definition['name'].lower(): function_definition['function'] for function_definition in function_definitions}

    stop=['Observation:']

    response = assistant.get_chat_response(prompt=prompt.format(prompt=user_prompt), stop=stop, **kwargs)
    clear_output(wait=True)
    print(response.choices[0].message.content.strip())

    # action loop
    next_message = ''
    counter = max_loops
    while True:
        if counter < max_loops:
            time.sleep(sleep)
        if len(response.choices[0].message.content.split('Final Answer:')) > 1:
            break
        action_result = parse_action_with_timer(response.choices[0].message.content, api_key=assistant.api_key, action_dict=action_dict, cache=cache, max_tokens=max_observation_tokens)
        if next_message != '':
            next_message = next_message + ' '
        next_message += response.choices[0].message.content.strip() + '\nObservation: ' + action_result + '\n'
        counter_ = min(max_loops, 10)
        while assistant.calculate_num_tokens(next_message) > memory_tokens:
            pattern = r'Thought:.*\nAction:.*\nAction Input:.*\nObservation:.*\n'

            # Remove the oldest entries using re.sub()
            next_message = re.sub(pattern, '', next_message, count=1)
            counter_ -= 1
            if counter_ == 0:
                next_message = truncate_text(assistant, next_message, memory_tokens, side='left')
                break
        # get the next action
        response = assistant.get_chat_response(prompt=None, stop=stop, inject_messages=[{0: {'role': 'user', 'content': user_prompt}}, {1: {'role': 'assistant', 'content': next_message}}], **kwargs)

        clear_output(wait=True)
        print(next_message + response.choices[0].message.content.strip())
        if counter == 0:
            break
        counter -= 1
        
    return response.choices[0].message.content.split('Final Answer:')[-1].strip(), cache

# Action Loop Setup

In [48]:
# example on how to create a function
def ask_chatgpt(last_action_input, chatgpt, max_tokens): # must include these inputs
    print('Asking ChatGPT: ' + last_action_input + '...') # print what you are doing
    response = chatgpt.get_chat_response(last_action_input) # processing logic goes here
    return truncate_text(chatgpt, response.choices[0].message.content.strip(), max_tokens) # end with a truncate_text call passing chatgpt, text, and max_tokens

In [49]:
# List of all functions/tools you want to give chatGPT access to
function_definitions = [
    {
        'name': 'Ask ChatGPT', # What you want chatGPT to see for the function name
        'definition': 'Ask ChatGPT a question or give ChatGPT a prompt.', # What you want chatGPT to see for the function definition
        'function': ask_chatgpt # the function you want to call
    },
    {
        'name': 'Web Search', 
        'definition': 'Searches the web for the given search query.', 
        'function': web_search 
    },
    {
        'name': 'Get Readable Content',
        'definition': 'Returns the readable content of the given url.',
        'function': get_readable_content
    },
    {
        'name': 'Get Internal Links',
        'definition': 'Returns the internal links of the given url.',
        'function': get_internal_links
    },
    {
        'name': 'Get External Links',
        'definition': 'Returns the external links of the given url.',
        'function': get_external_links
    },
    {
        'name': 'Run Python Code',
        'definition': 'Runs the given Python code.',
        'function': run_python_code
    },
]

# Run Action Loop

In [63]:
# Example
latest_news_summary, cache = action_loop(assistant, user_prompt="""Find the latest in ai news and then give me a 500-word summary. No python code allowed.
Final Answer must be in the following format:
Final Answer: <h1>...</h1>
<p>...</p>
<h2>...</h2>
<p>...</p>
...""", function_definitions=function_definitions, cache=cache, temperature=0.9)

Thought: I need to find a reliable source for AI news and then summarize it in my own words.
Action: Web Search
Action Input: "latest news in AI"
Observation: {title: Artificial Intelligence News -- ScienceDaily,
	body: AI Technology Generates Original Proteins from Scratch Jan. 26, 2023 — Scientists have created an AI system capable of generating artificial enzymes from scratch. In laboratory tests, some of...,
	url: https://www.sciencedaily.com/news/computers_math/artificial_intelligence/,
	title: 3 AI Stocks You Cant Afford to Miss | Nasdaq,
	body: The companys five-year compound annual growth rate on the top line stands at 34.9%. The latest earnings report showed a somewhat slower year-over-year revenue increase of 27%, but thats still a ...,
	url: https://www.nasdaq.com/articles/3-ai-stocks-you-cant-afford-to-miss,
	title: Artificial intelligence | MIT News | Massachusetts Institute of Technology,
	body: Integrating humans with AI in structural design A process that seeks feedback

In [64]:
print(latest_news_summary)

<h1>Latest AI News Summary</h1>
<p>AI researchers have developed an AI system capable of creating artificial enzymes from scratch. The system can generate original proteins without using existing templates or structures, opening up new possibilities for drug discovery and industrial biotechnology.</p>
<p>Three AI stocks have been identified by Nasdaq as must-buys. The list includes Nvidia, Alphabet and Amazon, with a focus on their potential to revolutionize industries like gaming, autonomous vehicles and e-commerce, respectively.</p>
<p>MIT News reports that AI systems perform better in tandem with human feedback than working alone. Integrating humans with AI in structural design has been shown to be more effective at optimization than fully automated systems.</p>
<p>Elon Musk has joined hundreds of technologists in calling for a six-month pause on training advanced AI systems. They warn that companies racing to develop ever more powerful AI systems could pose a significant risk to so